<a href="https://colab.research.google.com/github/VaidehiBisen14/DEEPLEARNINGVaidehi/blob/main/practno_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
RNN (LSTM) classifier for breast tumor vs normal.

Assumptions:
- Dataset at ./Process_Data.csv
- The script attempts to auto-detect the target column (looks for common names or any binary column).
- If non-numeric feature columns are present they will be dropped (you can add encoding).
- Uses timesteps=1 (features are provided as a single timestep vector) — common for tabular data when using RNN/LSTM.
"""

import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

DATA_PATH = "Process_Data.csv"   # adjust if needed

def load_and_inspect(path):
    df = pd.read_csv(path)
    print("Loaded:", path)
    print("Shape:", df.shape)
    print("Columns:", df.columns.tolist())
    print(df.head())
    return df

def detect_target(df):
    # Try common target names first
    possible_targets = ['target', 'label', 'class', 'Tissue', 'tissue', 'diagnosis', 'Diagnosis', 'Outcome', 'outcome']
    for name in possible_targets:
        if name in df.columns:
            return name
    # otherwise pick a column with exactly two unique values
    for c in df.columns:
        if df[c].nunique() == 2:
            return c
    # fallback: try to find a numeric column with small unique count
    for c in df.columns:
        if pd.api.types.is_integer_dtype(df[c]) and df[c].nunique() <= 10 and df[c].nunique() != df.shape[0]:
            return c
    raise ValueError("Could not auto-detect a binary target column. Please provide the target column name.")

def prepare_data(df, target_col):
    X = df.drop(columns=[target_col]).copy()
    y = df[target_col].copy()

    # Drop purely non-numeric columns for simplicity (you can add encoding here)
    non_numeric = X.select_dtypes(include=['object', 'category']).columns.tolist()
    if non_numeric:
        print("Dropping non-numeric columns (for this demo):", non_numeric)
        X = X.drop(columns=non_numeric)

    # Fill NaNs
    if X.isna().sum().sum() > 0:
        print("Filling missing feature values with column mean.")
        X = X.fillna(X.mean())

    # Encode target to 0/1
    if y.dtype == 'object' or y.dtype.name == 'category':
        le = LabelEncoder()
        y_enc = le.fit_transform(y)
        classes = le.classes_.tolist()
    else:
        le = None
        y_enc = LabelEncoder().fit_transform(y) if y.nunique() <= 10 else y.values
        classes = np.unique(y_enc).tolist()

    return X.values, y_enc, X.columns.tolist(), classes, le

def build_model(timesteps, n_features):
    model = Sequential([
        LSTM(64, input_shape=(timesteps, n_features), return_sequences=False),
        Dropout(0.3),
        Dense(32, activation='relu'),
        Dropout(0.2),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def main():
    df = load_and_inspect(DATA_PATH)
    target_col = detect_target(df)
    print("Detected target column:", target_col)
    print("Target distribution:\n", df[target_col].value_counts())

    X, y, feature_names, classes, label_encoder = prepare_data(df, target_col)

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    # Scaling
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # RNN shape: (samples, timesteps, features). We'll use timesteps=1
    timesteps = 1
    n_features = X_train_scaled.shape[1]
    X_train_rnn = X_train_scaled.reshape((X_train_scaled.shape[0], timesteps, n_features))
    X_test_rnn = X_test_scaled.reshape((X_test_scaled.shape[0], timesteps, n_features))
    print("RNN input shape:", X_train_rnn.shape)

    model = build_model(timesteps, n_features)
    print(model.summary())

    callbacks = [
        EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True),
        ModelCheckpoint("best_rnn_model.h5", monitor='val_loss', save_best_only=True)
    ]

    history = model.fit(
        X_train_rnn, y_train,
        validation_split=0.15,
        epochs=100,
        batch_size=32,
        callbacks=callbacks,
        verbose=2
    )

    # Evaluate
    loss, acc = model.evaluate(X_test_rnn, y_test, verbose=0)
    print(f"Test loss: {loss:.4f}, Test accuracy: {acc:.4f}")

    # Predictions
    y_prob = model.predict(X_test_rnn).ravel()
    y_pred = (y_prob >= 0.5).astype(int)

    print("Classification report:")
    print(classification_report(y_test, y_pred, target_names=[str(c) for c in classes]))
    print("Confusion matrix:\n", confusion_matrix(y_test, y_pred))

    # Plot history
    plt.figure(figsize=(10,4))
    plt.subplot(1,2,1)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title("Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend(['train','val'])

    plt.subplot(1,2,2)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title("Accuracy")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.legend(['train','val'])
    plt.tight_layout()
    plt.show()

    # Save model & scaler
    model.save("rnn_breast_model.h5")
    np.save("scaler_mean.npy", scaler.mean_)
    np.save("scaler_scale.npy", scaler.scale_)
    print("Saved rnn_breast_model.h5 and scaler params.")

    # Show sample predictions
    sample_df = pd.DataFrame({
        'pred_prob': y_prob[:10],
        'pred_label': y_pred[:10],
        'true_label': y_test[:10]
    })
    print("\nSample predictions (first 10):\n", sample_df)

if __name__ == "__main__":
    main()


Loaded: Process_Data.csv
Shape: (100000, 21)
Columns: ['Bill_ID', 'Client_ID', 'State', 'Original_Charge', 'Reduction', 'First_Activity', 'Last_Activity', 'Team_1_Start', 'Team_1_End', 'Team_2_Start', 'Team_2_End', 'Team_3_Start', 'Team_3_End', 'Completed', 'Closed', 'Type_A', 'Type_B', 'Type_C', 'Type_D', 'Type_E', 'Type_F']
   Bill_ID  Client_ID State  Original_Charge  Reduction    First_Activity  \
0   230000     101234    VA            47.00      47.00   11/15/2018 0:15   
1   230001     208827    CA           371.00     272.19  11/15/2018 22:07   
2   230002      93156    AZ           337.67      35.96   11/21/2018 6:19   
3   230003     174724    NY           194.35     194.35    12/3/2018 7:50   
4   230004      92975    MI           200.00      61.22   12/12/2018 3:52   

      Last_Activity      Team_1_Start        Team_1_End     Team_2_Start  ...  \
0  11/22/2018 17:04   11/15/2018 0:15   11/15/2018 0:16  11/19/2018 8:24  ...   
1    12/3/2018 7:44  11/15/2018 22:07  11/15/20

/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 64)             │        18,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,057 (82.25 KB)

...

2125/2125 - 7s - 3ms/step - accuracy: 0.9704 - loss: 0.1114 - val_accuracy: 0.9716 - val_loss: 0.1029
Epoch 10/100
2125/2125 - 10s - 5ms/step - accuracy: 0.9704 - loss: 0.1109 - val_accuracy: 0.9717 - val_loss: 0.1029
Epoch 11/100


2125/2125 - 10s - 5ms/step - accuracy: 0.9704 - loss: 0.1099 - val_accuracy: 0.9720 - val_loss: 0.1026
Epoch 12/100
2125/2125 - 7s - 3ms/step - accuracy: 0.9704 - loss: 0.1097 - val_accuracy: 0.9717 - val_loss: 0.1030
Epoch 13/100
2125/2125 - 6s - 3ms/step - accuracy: 0.9704 - loss: 0.1103 - val_accuracy: 0.9721 - val_loss: 0.1028
Epoch 14/100


2125/2125 - 7s - 3ms/step - accuracy: 0.9705 - loss: 0.1087 - val_accuracy: 0.9719 - val_loss: 0.1021
Epoch 15/100
2125/2125 - 6s - 3ms/step - accuracy: 0.9706 - loss: 0.1092 - val_accuracy: 0.9721 - val_loss: 0.1023
Epoch 16/100


2125/2125 - 7s - 3ms/step - accuracy: 0.9705 - loss: 0.1089 - val_accuracy: 0.9722 - val_loss: 0.1021
Epoch 17/100
2125/2125 - 10s - 5ms/step - accuracy: 0.9705 - loss: 0.1086 - val_accuracy: 0.9717 - val_loss: 0.1026
Epoch 18/100
2125/2125 - 9s - 4ms/step - accuracy: 0.9705 - loss: 0.1086 - val_accuracy: 0.9719 - val_loss: 0.1024
Epoch 19/100
2125/2125 - 7s - 3ms/step - accuracy: 0.9707 - loss: 0.1083 - val_accuracy: 0.9717 - val_loss: 0.1030
Epoch 20/100
2125/2125 - 6s - 3ms/step - accuracy: 0.9707 - loss: 0.1080 - val_accuracy: 0.9719 - val_loss: 0.1024
Epoch 21/100


2125/2125 - 10s - 5ms/step - accuracy: 0.9706 - loss: 0.1082 - val_accuracy: 0.9721 - val_loss: 0.1018
Epoch 22/100
